In [1]:
# -*- encoding=utf-8 -*-
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from tensorflow.python.ops import math_ops
from keras.applications.vgg16 import preprocess_input
import keras

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

path = './data/'

import h5py

batchsize = 32
image_size = (299,299)
lambda_func = preprocess_input

width = image_size[0]
height = image_size[1]
input_tensor = Input((height, width, 3))
x = input_tensor
if lambda_func:
    x = Lambda(lambda_func)(x)

base_model = VGG16(input_tensor=x, weights='imagenet', include_top=False)

# 先fine-tune全连接层
for layer in base_model.layers[:]:
    layer.trainable = False

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='tanh')(x)
predictions = Dense(30, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


for i, layer in enumerate(base_model.layers):
    print(i, layer.name)




gen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

gen2 = ImageDataGenerator(rescale=1./255)

train_generator = gen.flow_from_directory(path+"image_train", image_size, shuffle=False,
                                          batch_size=batchsize,class_mode='categorical')
validation_generator = gen2.flow_from_directory(path+"image_val", image_size, shuffle=False,
                                         batch_size=batchsize,class_mode='categorical')


# train = model.predict_generator(train_generator,100)
# test = model.predict_generator(test_generator)

# log_loss
def log_loss(y_true, y_pred):
    epsilon = 1e-7
    predictions = math_ops.to_float(y_pred)
    labels = math_ops.to_float(y_true)
    predictions.get_shape().assert_is_compatible_with(labels.get_shape())
    return -(math_ops.reduce_sum(math_ops.multiply(labels,math_ops.log(predictions)+epsilon)))/batchsize


model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy',
              metrics=[log_loss])

model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.samples,
            nb_epoch=50,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.samples/batchsize)


# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in base_model.layers[18:]:
    layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy',metrics=[log_loss])

model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.samples,
            nb_epoch=50,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.samples/batchsize)


model.save_weights('model/Vgg16_e50.h5')

Using TensorFlow backend.


(0, 'input_1')
(1, 'lambda_1')
(2, 'block1_conv1')
(3, 'block1_conv2')
(4, 'block1_pool')
(5, 'block2_conv1')
(6, 'block2_conv2')
(7, 'block2_pool')
(8, 'block3_conv1')
(9, 'block3_conv2')
(10, 'block3_conv3')
(11, 'block3_pool')
(12, 'block4_conv1')
(13, 'block4_conv2')
(14, 'block4_conv3')
(15, 'block4_pool')
(16, 'block5_conv1')
(17, 'block5_conv2')
(18, 'block5_conv3')
(19, 'block5_pool')
Found 4260 images belonging to 30 classes.
Found 1080 images belonging to 30 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:93: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:93: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=133, epochs=50, validation_steps=33)`


Epoch 1/50
133/133 [==============================] - 114s 856ms/step - loss: 3.6932 - log_loss: 3.6703 - val_loss: 3.4270 - val_log_loss: 3.4270
Epoch 2/50
133/133 [==============================] - 112s 842ms/step - loss: 3.5174 - log_loss: 3.4986 - val_loss: 3.4124 - val_log_loss: 3.4124
Epoch 3/50
133/133 [==============================] - 111s 833ms/step - loss: 3.5168 - log_loss: 3.4968 - val_loss: 3.4289 - val_log_loss: 3.4289
Epoch 4/50
133/133 [==============================] - 111s 838ms/step - loss: 3.5176 - log_loss: 3.4953 - val_loss: 3.4093 - val_log_loss: 3.4093
Epoch 5/50
133/133 [==============================] - 110s 828ms/step - loss: 3.5208 - log_loss: 3.4992 - val_loss: 3.4129 - val_log_loss: 3.4129
Epoch 6/50
133/133 [==============================] - 111s 835ms/step - loss: 3.5062 - log_loss: 3.4849 - val_loss: 3.4168 - val_log_loss: 3.4168
Epoch 7/50
133/133 [==============================] - 112s 839ms/step - loss: 3.5170 - log_loss: 3.5170 - val_loss: 3.4067 -

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:114: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:114: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=133, epochs=50, validation_steps=33)`


133/133 [==============================] - 113s 847ms/step - loss: 3.4987 - log_loss: 3.4770 - val_loss: 3.3982 - val_log_loss: 3.3982
Epoch 2/50
133/133 [==============================] - 111s 838ms/step - loss: 3.4129 - log_loss: 3.3909 - val_loss: 3.4018 - val_log_loss: 3.4018
Epoch 3/50
133/133 [==============================] - 112s 841ms/step - loss: 14.5533 - log_loss: 27.6473 - val_loss: 15.5686 - val_log_loss: 30.0441
Epoch 4/50
133/133 [==============================] - 110s 830ms/step - loss: 15.5803 - log_loss: 29.7537 - val_loss: 15.5686 - val_log_loss: 30.0441
Epoch 5/50
133/133 [==============================] - 109s 820ms/step - loss: 15.5803 - log_loss: 29.8467 - val_loss: 15.5686 - val_log_loss: 30.0441
Epoch 6/50
133/133 [==============================] - 109s 821ms/step - loss: 15.5803 - log_loss: 29.8871 - val_loss: 15.5686 - val_log_loss: 30.0441
Epoch 7/50
133/133 [==============================] - 108s 812ms/step - loss: 15.5803 - log_loss: 29.8915 - val_loss: 1

KeyboardInterrupt: 